In [1]:
%%writefile coalesced.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Task 1.1: Coalesced Access
// Threads access memory consecutively: Thread i reads Data[i]
__global__ void coalesced_kernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Coalesced read and write
        data[idx] = data[idx] * 2.0f;
    }
}

int main() {
    int n = 1 << 24; // 16M floats
    size_t bytes = n * sizeof(float);
    float *d_data;

    cudaMalloc(&d_data, bytes);

    // Warmup
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    coalesced_kernel<<<gridSize, blockSize>>>(d_data, n);
    cudaDeviceSynchronize();

    // Timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start); cudaEventCreate(&stop);

    cudaEventRecord(start);
    coalesced_kernel<<<gridSize, blockSize>>>(d_data, n);
    cudaEventRecord(stop);

    cudaDeviceSynchronize();
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    printf("Coalesced Access Time: %.3f ms\n", milliseconds);
    printf("Effective Bandwidth: %.2f GB/s\n", (2.0f * bytes) / (milliseconds * 1e6));

    cudaFree(d_data);
    return 0;
}

Writing coalesced.cu


In [2]:
%%writefile non_coalesced.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Task 1.2: Non-Coalesced Access (Strided)
// Threads access memory with a stride: Thread i reads Data[i * stride]
__global__ void strided_kernel(float* data, int n, int stride) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    // Bounds check accounts for the stride to prevent out-of-bounds
    if (idx * stride < n) {
        // Strided read and write (Performance Killer)
        data[idx * stride] = data[idx * stride] * 2.0f;
    }
}

int main() {
    int n = 1 << 24; // 16M floats
    size_t bytes = n * sizeof(float);
    float *d_data;
    int stride = 32; // Stride of 32 breaks coalescing completely

    cudaMalloc(&d_data, bytes);

    // Kernel configuration
    int blockSize = 256;
    // Grid must cover the *indices*, not the total array size
    int num_elements_accessed = n / stride;
    int gridSize = (num_elements_accessed + blockSize - 1) / blockSize;

    // Timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start); cudaEventCreate(&stop);

    cudaEventRecord(start);
    strided_kernel<<<gridSize, blockSize>>>(d_data, n, stride);
    cudaEventRecord(stop);

    cudaDeviceSynchronize();
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    printf("Non-Coalesced (Stride %d) Time: %.3f ms\n", stride, milliseconds);
    // Note: We transfer 32x less data, but check the bandwidth utilization!
    size_t active_bytes = (n / stride) * sizeof(float);
    printf("Effective Bandwidth: %.2f GB/s\n", (2.0f * active_bytes) / (milliseconds * 1e6));

    cudaFree(d_data);
    return 0;
}

Writing non_coalesced.cu


In [3]:
%%writefile shared_memory.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>

#define TILE_WIDTH 32

// Baseline: Global Memory Only
__global__ void matmul_naive(float *A, float *B, float *C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < N; k++) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

// Optimized: Shared Memory Tiling
__global__ void matmul_shared(float *A, float *B, float *C, int N) {
    __shared__ float tile_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float tile_B[TILE_WIDTH][TILE_WIDTH];

    int row = blockIdx.y * TILE_WIDTH + threadIdx.y;
    int col = blockIdx.x * TILE_WIDTH + threadIdx.x;
    float sum = 0.0f;

    for (int t = 0; t < (N + TILE_WIDTH - 1) / TILE_WIDTH; t++) {
        int col_A = t * TILE_WIDTH + threadIdx.x;
        int row_B = t * TILE_WIDTH + threadIdx.y;

        // Load A tile
        if (row < N && col_A < N)
            tile_A[threadIdx.y][threadIdx.x] = A[row * N + col_A];
        else
            tile_A[threadIdx.y][threadIdx.x] = 0.0f;

        // Load B tile
        if (row_B < N && col < N)
            tile_B[threadIdx.y][threadIdx.x] = B[row_B * N + col];
        else
            tile_B[threadIdx.y][threadIdx.x] = 0.0f;

        __syncthreads(); // Wait for load

        for (int k = 0; k < TILE_WIDTH; k++) {
            sum += tile_A[threadIdx.y][k] * tile_B[k][threadIdx.x];
        }

        __syncthreads(); // Wait for compute before next load
    }

    if (row < N && col < N) {
        C[row * N + col] = sum;
    }
}

int main() {
    int N = 2048; // Large enough to see difference
    size_t bytes = N * N * sizeof(float);
    printf("Matrix Multiplication (N=%d)\n", N);

    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, bytes);
    cudaMalloc(&d_B, bytes);
    cudaMalloc(&d_C, bytes);

    // Initialize random data (omitted for brevity, assume populated)

    dim3 block(TILE_WIDTH, TILE_WIDTH);
    dim3 grid((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) / TILE_WIDTH);

    // Timing Naive
    cudaEvent_t start, stop;
    cudaEventCreate(&start); cudaEventCreate(&stop);

    cudaEventRecord(start);
    matmul_naive<<<grid, block>>>(d_A, d_B, d_C, N);
    cudaEventRecord(stop);
    cudaDeviceSynchronize();

    float naive_time = 0;
    cudaEventElapsedTime(&naive_time, start, stop);
    printf("Naive Time: %.3f ms\n", naive_time);

    // Timing Shared
    cudaEventRecord(start);
    matmul_shared<<<grid, block>>>(d_A, d_B, d_C, N);
    cudaEventRecord(stop);
    cudaDeviceSynchronize();

    float shared_time = 0;
    cudaEventElapsedTime(&shared_time, start, stop);
    printf("Shared Time: %.3f ms\n", shared_time);

    printf("Speedup: %.2fx\n", naive_time / shared_time);

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    return 0;
}

Writing shared_memory.cu


In [4]:
%%writefile cpu_baseline.py
import time
import numpy as np

def run_cpu_baseline():
    # NPU Simulation Parameters (Week 1 Project)
    # LLaMA-sized layer
    M = 1       # Batch size (Edge Inference)
    K = 4096    # Input Features
    N = 4096    # Output Features

    print(f"Running CPU Baseline for Edge AI NPU Simulation...")
    print(f"Dimensions: Input({M}x{K}) * Weights({K}x{N})")

    # Initialize data
    X = np.random.randn(M, K).astype(np.float32)
    W = np.random.randn(K, N).astype(np.float32)

    # Measure Runtime
    start_time = time.perf_counter()

    # The actual NPU Operation
    Y = np.dot(X, W)
    Y = np.maximum(Y, 0) # ReLU

    end_time = time.perf_counter()
    duration_ms = (end_time - start_time) * 1000

    print(f"CPU Runtime: {duration_ms:.4f} ms")
    return duration_ms

if __name__ == "__main__":
    run_cpu_baseline()

Writing cpu_baseline.py


In [5]:

!nvcc -arch=sm_75 -o coalesced coalesced.cu
!./coalesced
!nvcc -arch=sm_75 -o non_coalesced non_coalesced.cu
!./non_coalesced


!nvcc -arch=sm_75 -o shared_memory shared_memory.cu
!./shared_memory


!python cpu_baseline.py

Coalesced Access Time: 0.583 ms
Effective Bandwidth: 230.10 GB/s
Non-Coalesced (Stride 32) Time: 0.565 ms
Effective Bandwidth: 7.43 GB/s
Matrix Multiplication (N=2048)
Naive Time: 53.718 ms
Shared Time: 41.965 ms
Speedup: 1.28x
Running CPU Baseline for Edge AI NPU Simulation...
Dimensions: Input(1x4096) * Weights(4096x4096)
CPU Runtime: 4.5674 ms
